In [24]:
!pip install praw nltk
# get credentioals from here https://www.reddit.com/prefs/apps/

import logging

# Set logging level to suppress PRAW warning messages
logging.getLogger("praw").setLevel(logging.WARNING)
logging.getLogger("praw").setLevel(logging.ERROR)

from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [25]:
import praw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re

# Reddit API credentials
reddit_client_id = 'bLfHRgL3R8y69_QUj7xMrA'
reddit_client_secret = 'HO0fvx3tIbnujuyCqIPmjXT9lopiCg'
reddit_user_agent = 'CommunicationTop2560'

# Initialize Reddit API
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Subreddit to follow
subreddit_name = 'gadgets'
subreddit = reddit.subreddit(subreddit_name)

# Extracting data from the subreddit
posts = []
for submission in subreddit.top('year', limit=10000):  # Example: Top posts of the month, limit to 1000
    posts.append({
        'ID': submission.id,
        'Title': submission.title,
        'Score': submission.score,
        'URL': submission.url,
        'Num_comments': submission.num_comments,
        'Created_utc': submission.created_utc,
        'Author':submission.author,
    })

# Create a DataFrame from the extracted data
df = pd.DataFrame(posts)

# Save the DataFrame to a CSV file
# df.to_csv('gadgets_data.csv', index=False)

# Display the DataFrame
df

,ID,Title,Score,URL,Num_comments,Created_utc,Author
0,10upla2,"Couple used Apple AirTag to track luggage, fou...",51227,https://www.pcmag.com/news/couple-used-apple-a...,1818,1.675636e+09,thebelsnickle1991
1,12x30gv,Scalpers are struggling to sell PlayStation 5 ...,47858,https://www.techspot.com/news/98403-scalpers-s...,2636,1.682310e+09,thebelsnickle1991
2,14ddlcs,EU: Smartphones Must Have User-Replaceable Bat...,36832,https://www.pcmag.com/news/eu-smartphones-must...,3161,1.687178e+09,FigAAAro_22
3,10w0qp4,The Switch has now outsold the PS4 and Game Bo...,31390,https://www.engadget.com/the-switch-has-outsol...,1813,1.675773e+09,chrisdh79
4,11c8ojr,Nokia is supporting a user's right-to-repair b...,29539,https://www.digitaltrends.com/mobile/hmd-globa...,1024,1.677398e+09,MicroSofty88
...,...,...,...,...,...,...,...
995,191n2zx,Samsung's 990 EVO SSD could support both PCIe ...,145,https://www.techspot.com/news/101439-samsung-9...,11,1.704726e+09,chrisdh79
996,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,143,https://www.macrumors.com/2023/02/28/iphone-16...,42,1.677591e+09,chrisdh79
997,184nrlt,AMD Radeon RX 7900M performance debut: Alienwa...,145,https://www.notebookcheck.net/AMD-Radeon-RX-79...,20,1.701040e+09,Stiven_Crysis
998,14f44hn,LCD TV sunset on the horizon as R&D ends and e...,143,https://www.techspot.com/news/99144-lcd-tv-sun...,26,1.687344e+09,chrisdh79


In [26]:
post = reddit.submission(id="14ddlcs")
comments = post.comments

for comment in comments[:10]:
  print("commentdata", comment.body)
  print("author", comment.author)

commentdata I, too, hail the not-quite-return of Nokia
author Rabatis
commentdata Hopefully soaking the adhesive under the battery with 3 liters of IPA will not be the manufacturers idea of a "User-replacabale" Battery.

Edit : IPA as in "Isopropyl alcohol" not "Inidan Pale Ale". Never realized they had a similar Abbreviation
author A_chilles
commentdata Bring back the damned removable storage ability too.  There's no reason I should have to upgrade phones just to get more storage.
author kickit256
commentdata I think it's crazy how polarizing this is. Often times, people feel that their phone needs upgrading because the battery isn't what it used to be. While this may lead to issues pertaining to form factor, it will also be a fantastic step towards straying away from rampant consumerism and reduce E-waste. I am very excited to see electronics manufacturers held to the same regard as vehicle manufacturers. Just because it is on a smaller scale doesn't mean it is proprietary.
author Dr

In [27]:
# Extracting comments from a specific post
post_id = "14ddlcs"  # Replace with the actual post ID
post = reddit.submission(id=post_id)
comments = post.comments

# Extracting more information from comments
comment_data = []
for comment in comments[:10]:
    comment_data.append({
        'Comment': comment.body,
        'Author': str(comment.author),
        'Score': comment.score,
        'Created_utc': comment.created_utc,
    })

# Create a DataFrame from the extracted comment data
comments_df = pd.DataFrame(comment_data)
comments_df

,Comment,Author,Score,Created_utc
0,"I, too, hail the not-quite-return of Nokia",Rabatis,774,1.687185e+09
1,Hopefully soaking the adhesive under the batte...,A_chilles,2548,1.687180e+09
2,Bring back the damned removable storage abilit...,kickit256,716,1.687207e+09
3,I think it's crazy how polarizing this is. Oft...,Dracekidjr,1461,1.687181e+09
4,We've gone full circle. It went from user swap...,IronhideD,436,1.687181e+09
5,The EU continue to do all the heavy lifting on...,Solitude_Dude,63,1.687229e+09
6,Hopefully phone companies will still be able t...,NizarNoor,284,1.687181e+09
7,I doubt this will really change anything becau...,ryzenguy111,114,1.687183e+09
8,If i can get smart phone with replaceable batt...,hbkdll,8,1.687242e+09
9,"This sounds great, but it's a bit of a moot ge...",steaminghotshiitake,63,1.687185e+09


In [28]:
# Extracting comments from posts with titles containing 'apple' or 'iphone'
filtered_posts = df[df['Title'].str.contains('iphone', case=False, regex=True)]
selected_post_ids = filtered_posts['ID'].tolist()
filtered_posts

,ID,Title,Score,URL,Num_comments,Created_utc,Author
9,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",24609,https://www.digitaltrends.com/mobile/apple-iph...,3097,1.673748e+09,ChickenTeriyakiBoy1
11,11283q1,50% rejection rate for iPhone casings produced...,22655,https://9to5mac.com/2023/02/14/iphone-casings-...,1565,1.676389e+09,SUPRVLLAN
20,16i2vz9,Apple users bash new iPhone 15: ‘Innovation di...,18716,https://nypost.com/2023/09/13/apple-users-bash...,4111,1.694649e+09,etfvpu
22,116cqrg,Leaked image appears to show iPhone 15 Pro wit...,18050,https://9to5mac.com/2023/02/16/iphone-15-pro-u...,2071,1.676818e+09,thebelsnickle1991
23,11vve8z,"Factory-Sealed Original iPhone Sells for $55,0...",17512,https://www.macrumors.com/2023/03/17/original-...,819,1.679255e+09,None
...,...,...,...,...,...,...,...
915,16muhcz,The iPhone 15 Pro and 15 Pro Max debut today w...,231,https://www.techspot.com/products/smartphones/...,441,1.695138e+09,chrisdh79
978,16736jn,iPhone 16 OLED Panels May Use Micro-Lens Techn...,161,https://www.macrumors.com/2023/09/01/iphone-16...,64,1.693563e+09,Avieshek
979,11b7zzj,Kuo: iPhone 15 Pro Models to Feature Improved ...,157,https://www.macrumors.com/2023/02/24/kuo-iphon...,18,1.677288e+09,None
981,16nha7g,We Are Retroactively Dropping the iPhone’s Rep...,155,https://www.ifixit.com/News/82493/we-are-retro...,17,1.695204e+09,chrisdh79


In [29]:
# Extracting comments from posts with titles containing 'iphone'
filtered_posts = df[df['Title'].str.contains('iphone', case=False, regex=True)]
selected_post_ids = filtered_posts['ID'].tolist()

iphone_comments_data = []

for post_id in selected_post_ids:
    try:
        post = reddit.submission(id=post_id)
        post_title = df[df['ID'] == post_id]['Title'].values[0]
        comments = post.comments.list()

        # Extracting more information from comments
        for comment in comments:
            if isinstance(comment, praw.models.MoreComments):
                continue  # Skip 'MoreComments' objects
            iphone_comments_data.append({
                'Post_ID': post_id,
                'Title': post_title,
                'Comment': comment.body,
                'Created_utc': comment.created_utc,
                'Author': str(comment.author) if comment.author else '[deleted]',
                'Upvote_ratio': post.upvote_ratio,
                'Score': post.score,
                'Subreddit': post.subreddit.display_name,
                'URL': post.url,
                'Domain': post.domain,
                'Media': post.media,
                'Is_video': post.is_video,
                'Thumbnail': post.thumbnail,
                'Permalink': post.permalink,
                'Selftext': post.selftext,
                'Upvotes': post.ups,
            })
    except Exception as e:
        print(f"Error processing post {post_id}: {str(e)}")

# Create a DataFrame from the extracted iPhone comments data
iphone_comments_df = pd.DataFrame(iphone_comments_data)
iphone_comments_df

,Post_ID,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Subreddit,URL,Domain,Media,Is_video,Thumbnail,Permalink,Selftext,Upvotes
0,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
1,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
2,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
3,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
4,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31108,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31109,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31110,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31111,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142


In [30]:
# # Extracting comments from posts with titles containing 'apple' or 'iphone'
# filtered_posts = df[df['Title'].str.contains('iphone', case=False, regex=True)]
# selected_post_ids = filtered_posts['ID'].tolist()

# iphone_comments_data = []

# for comment in comments:
#     if isinstance(comment, praw.models.MoreComments):
#         continue  # Skip 'MoreComments' objects
#     comment_text = comment.body

#     # Extract hashtags from the comment using a regular expression
#     hashtags = re.findall(r'#\w+', comment_text)

#     iphone_comments_data.append({
#         'Post_ID': post_id,
#         'Title': post_title,
#         'Comment': comment_text,
#         'Author': str(comment.author),
#         'Score': comment.score,
#         'Created_utc': comment.created_utc,
#         'Hashtags': hashtags,  # Add a new 'Hashtags' column
#     })
#     except Exception as e:
#         print(f"Error processing post {post_id}: {str(e)}")

# # Create a DataFrame from the extracted iPhone comments data
# iphone_comments_df1 = pd.DataFrame(iphone_comments_data)
# iphone_comments_df1

In [31]:
iphone_comments_df

,Post_ID,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Subreddit,URL,Domain,Media,Is_video,Thumbnail,Permalink,Selftext,Upvotes
0,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
1,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
2,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
3,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
4,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31108,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31109,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31110,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142
31111,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142


In [32]:
# Downloading the datasets
# df.to_csv('gadgets_data.csv', index=False)
# iphone_comments_df.to_csv('iphone_comments_data.csv', index=False)

In [33]:
# Combine Title and Selftext for text analysis
iphone_comments_df['Text'] = iphone_comments_df['Title'] + ' ' + iphone_comments_df['Comment'] + ' ' + iphone_comments_df['Selftext']

# Tokenize and clean the text
stop_words = set(stopwords.words('english'))
iphone_comments_df['Tokens'] = iphone_comments_df['Text'].apply(lambda x: word_tokenize(re.sub(r'[^a-zA-Z0-9]', ' ', str(x).lower())))
iphone_comments_df['Tokens'] = iphone_comments_df['Tokens'].apply(lambda x: [word for word in x if word.isalnum() and word not in stop_words])

# Flatten the list of tokens
all_tokens = [token for sublist in iphone_comments_df['Tokens'].tolist() for token in sublist]

# Identify keywords and hashtags using frequency distribution
fdist = FreqDist(all_tokens)
keywords = [word for word, freq in fdist.most_common(20) if len(word) > 2]

# Identify influencers (top authors)
top_authors = iphone_comments_df['Author'].value_counts().index[:5].tolist()

# Identify different groups (subreddits)
different_groups = iphone_comments_df['Subreddit'].unique().tolist()

# Display the results
print("Keywords and Hashtags:", keywords)
print("\nTop Influencers:", top_authors)
print("\nDifferent Groups (Subreddits):", different_groups)

Keywords and Hashtags: ['iphone', 'apple', 'pro', 'usb', 'new', 'phone', 'iphones', 'like', 'people', 'charging', 'max', 'port', 'battery', 'year', 'get', 'android', 'use', 'would']

Top Influencers: ['[deleted]', 'nicuramar', 'SUPRVLLAN', 'Sylvurphlame', 'Pubelication']

Different Groups (Subreddits): ['gadgets']


In [34]:
iphone_comments_df

,Post_ID,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Subreddit,URL,Domain,Media,Is_video,Thumbnail,Permalink,Selftext,Upvotes,Text,Tokens
0,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
1,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
2,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
3,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
4,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24611,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/M7OMipcAt8f6i...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24611,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31108,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31109,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31110,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,142,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31111,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,142,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD7

In [35]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [36]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

In [38]:
comments_df['Processed_Comment'] = iphone_comments_df['Comment'].apply(preprocess_text)

In [39]:
from collections import Counter

# Combine all processed comments into a single list of words
all_words = [word for sublist in comments_df['Processed_Comment'] for word in sublist]

# Get the most common keywords
common_keywords = Counter(all_words).most_common(10)
print("Common Keywords:", common_keywords)


Common Keywords: [('apple', 5), ('feature', 5), ('removal', 3), ('like', 2), ('iphone', 2), ('article', 2), ('idea', 2), ('everybody', 2), ('hates', 2), ('competitors', 2)]


In [40]:
influencers = comments_df['Author'].value_counts().head(5)
print("Top Influencers:", influencers)


Top Influencers: Rabatis       1
A_chilles     1
kickit256     1
Dracekidjr    1
IronhideD     1
Name: Author, dtype: int64


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Convert processed comments to a list of strings
comment_strings = [' '.join(comment) for comment in comments_df['Processed_Comment']]

# Vectorize the comments
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(comment_strings)

# Cluster comments using KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
comments_df['Cluster'] = kmeans.fit_predict(X)

# Display clusters
print("Clusters:")
for cluster_num in range(3):
    cluster_comments = comments_df[comments_df['Cluster'] == cluster_num]['Comment']
    print(f"Cluster {cluster_num + 1}:\n{cluster_comments}\n")

Clusters:
Cluster 1:
0           I, too, hail the not-quite-return of Nokia
1    Hopefully soaking the adhesive under the batte...
2    Bring back the damned removable storage abilit...
5    The EU continue to do all the heavy lifting on...
7    I doubt this will really change anything becau...
Name: Comment, dtype: object

Cluster 2:
3    I think it's crazy how polarizing this is. Oft...
8    If i can get smart phone with replaceable batt...
Name: Comment, dtype: object

Cluster 3:
4    We've gone full circle. It went from user swap...
6    Hopefully phone companies will still be able t...
9    This sounds great, but it's a bit of a moot ge...
Name: Comment, dtype: object



# expreiments below

In [ ]:
import praw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re

# Reddit API credentials
reddit_client_id = 'bLfHRgL3R8y69_QUj7xMrA'
reddit_client_secret = 'HO0fvx3tIbnujuyCqIPmjXT9lopiCg'
reddit_user_agent = 'CommunicationTop2560'

# Initialize Reddit API
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Subreddit to follow
subreddit_name = 'gadgets'
subreddit = reddit.subreddit(subreddit_name)

# Extracting data from the subreddit
posts = []
for submission in subreddit.top('year', limit=10000):
    posts.append({
        'ID': submission.id,
        'Title': submission.title,
        'Score': submission.score,
        'URL': submission.url,
        'Num_comments': submission.num_comments,
        'Created_utc': submission.created_utc,
        'Author': submission.author.name if submission.author else '[deleted]',
        'Upvote_ratio': submission.upvote_ratio,
        'Gilded': submission.gilded,
        'Over_18': submission.over_18,
        'Is_original_content': submission.is_original_content,
        'Is_self': submission.is_self,
        'Link_flair_text': submission.link_flair_text,
        'Link_flair_css_class': submission.link_flair_css_class,
        'Locked': submission.locked,
        'Stickied': submission.stickied,
        'Subreddit': submission.subreddit.display_name,
        # 'Subreddit_id': submission.subreddit.id,
        # 'Subreddit_subscribers': submission.subreddit.subscribers,
        # 'Media_embed': submission.media_embed,
        'Secure_media': submission.secure_media,
        'Secure_media_embed': submission.secure_media_embed,
        'Subreddit_type': submission.subreddit.subreddit_type,
        'Submission_type': submission.subreddit.submission_type,
        'Upvote_ratio': submission.upvote_ratio,
        'Domain': submission.domain,
        'Is_video': submission.is_video,
        'Thumbnail': submission.thumbnail,
        'Media_only': submission.media_only,
        'Is_crosspostable': submission.is_crosspostable,
        'Selftext_html': submission.selftext_html,
        'Distinguished': submission.distinguished,
        'Pinned': submission.pinned,
        'Visited': submission.visited,
        'Clicked': submission.clicked,
        'Hidden': submission.hidden,
        'Likes': submission.likes,
        'Is_reddit_media_domain': submission.is_reddit_media_domain,
        'Can_mod_post': submission.can_mod_post,
        'Banned_by': submission.banned_by,
        'Removed_by': submission.removed_by,
        'Edited': submission.edited,
        'Spoiler': submission.spoiler,
        'Num_reports': submission.num_reports,
        'Total_awards_received': submission.total_awards_received,
        'Author_flair_text': submission.author_flair_text,
        'Author_flair_css_class': submission.author_flair_css_class,
        'Permalink': submission.permalink,
        'Whitelist_status': submission.whitelist_status,
        'Content_categories': submission.content_categories,
        'Removal_reason': submission.removal_reason,
        'Selftext': submission.selftext,
        'Saved': submission.saved,
        'Is_meta': submission.is_meta,
        'Created': submission.created,
        'Is_robot_indexable': submission.is_robot_indexable,
        'Media': submission.media,
        'Secure_media_embed': submission.secure_media_embed,
        'Secure_media': submission.secure_media,
        'Num_crossposts': submission.num_crossposts,
        'All_awardings': submission.all_awardings,
        'Can_gild': submission.can_gild,
        'Category': submission.category,
        'Contest_mode': submission.contest_mode,
        'Downs': submission.downs,
        'Hide_score': submission.hide_score,
        'Is_crosspostable': submission.is_crosspostable,
        'Is_video': submission.is_video,
        'Link_flair_background_color': submission.link_flair_background_color,
        'Link_flair_css_class': submission.link_flair_css_class,
        'Link_flair_richtext': submission.link_flair_richtext,
        'Likes': submission.likes,
        'Media_only': submission.media_only,
        'Num_duplicates': submission.num_duplicates,
        'Num_reports': submission.num_reports,
        'Over_18': submission.over_18,
        'Removal_reason': submission.removal_reason,
        'Secure_media': submission.secure_media,
        'Secure_media_embed': submission.secure_media_embed,
        'Selftext': submission.selftext,
        'Selftext_html': submission.selftext_html,
        'Spoiler': submission.spoiler,
        'Subreddit': submission.subreddit,
        'Subreddit_subscribers': submission.subreddit.subscribers,
        'Subreddit_type': submission.subreddit.subreddit_type,
        'Submission_type': submission.subreddit.submission_type,
        'Thumbnail': submission.thumbnail,
        'Title': submission.title,
        'Total_awards_received': submission.total_awards_received,
        'Upvotes': submission.ups,
        'Url': submission.url,
        'Visited': submission.visited,
        'Whitelist_status': submission.whitelist_status,
    })

# Create a DataFrame from the extracted data
df = pd.DataFrame(posts)

# Display the DataFrame
df

In [ ]:
import praw
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Reddit API credentials
reddit_client_id = 'bLfHRgL3R8y69_QUj7xMrA'
reddit_client_secret = 'HO0fvx3tIbnujuyCqIPmjXT9lopiCg'
reddit_user_agent = 'CommunicationTop2560'

# Initialize Reddit API
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Subreddit to follow
subreddit_name = 'gadgets'
subreddit = reddit.subreddit(subreddit_name)

# Extracting data from the subreddit
posts = []
for submission in subreddit.top('year', limit=10000):
    posts.append({
        'ID': submission.id,
        'Title': submission.title,
        'Num_comments': submission.num_comments,
        'Author': submission.author.name if submission.author else '[deleted]',
        'Upvote_ratio': submission.upvote_ratio,
        'Subreddit': submission.subreddit.display_name,
        'URL': submission.url,
        'Domain': submission.domain,
        'Media': submission.media,
        'Is_video': submission.is_video,
        'Thumbnail': submission.thumbnail,
        'Permalink': submission.permalink,
        'Selftext': submission.selftext,
        'Upvotes': submission.ups,
    })

# Create a DataFrame from the extracted data
df = pd.DataFrame(posts)

# Combine Title and Selftext for text analysis
df['Text'] = df['Title'] + ' ' + df['Selftext']

# Tokenize and clean the text
stop_words = set(stopwords.words('english'))
df['Tokens'] = df['Text'].apply(lambda x: word_tokenize(re.sub(r'[^a-zA-Z0-9]', ' ', str(x).lower())))
df['Tokens'] = df['Tokens'].apply(lambda x: [word for word in x if word.isalnum() and word not in stop_words])

# Flatten the list of tokens
all_tokens = [token for sublist in df['Tokens'].tolist() for token in sublist]

# Identify keywords and hashtags using frequency distribution
fdist = FreqDist(all_tokens)
keywords = [word for word, freq in fdist.most_common(20) if len(word) > 2]

# Identify influencers (top authors)
top_authors = df['Author'].value_counts().index[:5].tolist()

# Identify different groups (subreddits)
different_groups = df['Subreddit'].unique().tolist()

# Display the results
print("Keywords and Hashtags:", keywords)
print("\nTop Influencers:", top_authors)
print("\nDifferent Groups (Subreddits):", different_groups)

In [ ]:
df